# Source

The `Source` class provides an interface and predefined methods for reading raw data from file and, if enabled, saving them in a condensed `hdf5` file.
The `evaluation` class should access the actual data for a deticated scan via `source.get_scan_data(scan_number)` or for a list of scans via `source.get_scan_list_data(scan_number_list)`.